In [1]:
# Imports

%cd /content/drive/MyDrive/video_classification_rnn

import os
import numpy as np
# import random
import cv2
# from google.colab.patches import cv2_imshow
import tensorflow as tf
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, SimpleRNN, GRU, LSTM, Dense, Flatten, TimeDistributed
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
# from models import RNN_model, GRU_model, LSTM_model, backbone_model
from load_video import load_video
from models import Models

/content/drive/.shortcut-targets-by-id/1enih2ED1voM3fLiLZJFTl-l1planNYI5/video_classification_rnn


In [2]:
# Parameters setting

frame_width = 80
frame_height = 80

batch_size = 4
epochs = 15

dataset_path = "/content/drive/MyDrive/dataset/joon_del"

In [3]:
# Create directories

if not os.path.exists("weights"):
    os.makedirs("weights")

### Preparing Data

In [4]:
# Count maximum number of frames in all videos

num_classes = os.listdir(dataset_path)
count_frames = []

for class_label in num_classes:
  videos = os.listdir(os.path.join(dataset_path, class_label))

  for video in videos:
    video_path = os.path.join(dataset_path, class_label, video)
    frames = cv2.VideoCapture(video_path)

    total_frames = int(frames.get(cv2.CAP_PROP_FRAME_COUNT))
    count_frames.append(total_frames)

max_seq_len = np.max(count_frames)
num_sampels = len(count_frames)

print("numsmaples: ", num_sampels)
print("max sequence lenght: ", max_seq_len)

numsmaples:  68
max sequence lenght:  316


In [5]:
def create_dataset():

  labels = []
  frames_dataset = []

  num_classes = os.listdir(dataset_path)
  frame_masks = np.zeros(shape=(num_sampels, max_seq_len), dtype="bool")
  video_count = 0

  for class_label in num_classes:
    videos = os.listdir(os.path.join(dataset_path, class_label))

    for video in videos:
      video_path = os.path.join(dataset_path, class_label, video)
      frames, num_frames = load_video(video_path, (frame_height, frame_width))

      padded_frames = np.zeros(shape=(max_seq_len, frame_height, frame_width, 3), dtype="float32")
      current_video_seq_len = min(max_seq_len, num_frames)

      # Normalize video frames
      for i, frame in enumerate(np.array(frames)):
          padded_frames[i, :] = frame / 255.

      frames_dataset.append(padded_frames)
      frame_masks[video_count, :current_video_seq_len] = 1

      video_count+=1

      # Create labels
      if class_label == "1" : 
        labels.append(1) 
      else: 
        labels.append(0)

  # Convert to np.array
  frames_dataset = np.array(frames_dataset)
  labels = np.array(labels)

  # Reshape labels
  labels = labels[..., np.newaxis]

  return [frames_dataset, frame_masks], labels

In [6]:
# Call data preparing function

X, Y = create_dataset()

In [7]:
# Splite data

X_train, X_val, mask_train, mask_val, Y_train, Y_val = train_test_split(X[0], X[1], Y, test_size = 0.2, random_state=42)

print("train data shape: ", X_train.shape, Y_train.shape, mask_train.shape, "validation data shape: ",  X_val.shape, mask_val.shape, Y_val.shape)

train data shape:  (54, 316, 80, 80, 3) (54, 1) (54, 316) validation data shape:  (14, 316, 80, 80, 3) (14, 316) (14, 1)


In [8]:
Y_val

array([[1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0]])

In [9]:
# Plot

# for frame in X_train[8]:
#     plt.subplot(2, 2, 1)
#     plt.imshow(frame[:, :, :], cmap="gray")
#     plt.show()

### Define Models, Compile and fit

01- RNN

In [10]:
models = Models((frame_height, frame_width), max_seq_len)
rnn_model = models.RNN_model()

rnn_model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

rnn_model.fit([X_train, mask_train], Y_train, validation_data=[[X_val, mask_val], Y_val], batch_size=batch_size, epochs=epochs)

Epoch 1/15
14/14 [==============================] - 68s 4s/step - loss: 0.9641 - accuracy: 0.4259 - val_loss: 0.8687 - val_accuracy: 0.5000
Epoch 2/15
14/14 [==============================] - 46s 3s/step - loss: 0.7801 - accuracy: 0.5556 - val_loss: 0.7631 - val_accuracy: 0.5000
Epoch 3/15
14/14 [==============================] - 45s 3s/step - loss: 0.7737 - accuracy: 0.4815 - val_loss: 0.8320 - val_accuracy: 0.5000
Epoch 4/15
14/14 [==============================] - 45s 3s/step - loss: 0.7863 - accuracy: 0.5000 - val_loss: 0.8008 - val_accuracy: 0.2857
Epoch 5/15
14/14 [==============================] - 45s 3s/step - loss: 0.7457 - accuracy: 0.5741 - val_loss: 0.7736 - val_accuracy: 0.5714
Epoch 6/15
14/14 [==============================] - 45s 3s/step - loss: 0.7063 - accuracy: 0.5741 - val_loss: 0.8396 - val_accuracy: 0.4286
Epoch 7/15
14/14 [==============================] - 45s 3s/step - loss: 0.6605 - accuracy: 0.6481 - val_loss: 0.7714 - val_accuracy: 0.4286
Epoch 8/15
14/14 [==

In [11]:
rnn_model.save_weights('/content/drive/MyDrive/video_classification_rnn/weights/rnn_model.h5')

In [12]:
rnn_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 80, 8  0           []                               
                                0, 3)]                                                            
                                                                                                  
 model (Functional)             (None, None, 10, 10  170416      ['input_1[0][0]']                
                                , 64)                                                             
                                                                                                  
 time_distributed_21 (TimeDistr  (None, None, 6400)  0           ['model[0][0]']                  
 ibuted)                                                                                    

#### 02- GRU

In [13]:
models = Models((frame_height, frame_width), max_seq_len)
gru_model = models.GRU_model()

gru_model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

gru_model.fit([X_train, mask_train], Y_train, validation_data=[[X_val, mask_val], Y_val], batch_size=3, epochs=epochs)


Epoch 1/15
18/18 [==============================] - 49s 2s/step - loss: 1.4271 - accuracy: 0.4630 - val_loss: 0.7630 - val_accuracy: 0.5714
Epoch 2/15
18/18 [==============================] - 32s 2s/step - loss: 0.7499 - accuracy: 0.5370 - val_loss: 0.8920 - val_accuracy: 0.4286
Epoch 3/15
18/18 [==============================] - 32s 2s/step - loss: 0.6923 - accuracy: 0.6296 - val_loss: 0.7960 - val_accuracy: 0.4286
Epoch 4/15
18/18 [==============================] - 32s 2s/step - loss: 0.7111 - accuracy: 0.6481 - val_loss: 0.7817 - val_accuracy: 0.5000
Epoch 5/15
18/18 [==============================] - 32s 2s/step - loss: 0.7401 - accuracy: 0.6111 - val_loss: 0.8839 - val_accuracy: 0.2143
Epoch 6/15
18/18 [==============================] - 32s 2s/step - loss: 0.6654 - accuracy: 0.6296 - val_loss: 0.8185 - val_accuracy: 0.3571
Epoch 7/15
18/18 [==============================] - 32s 2s/step - loss: 0.6763 - accuracy: 0.6667 - val_loss: 0.7306 - val_accuracy: 0.5714
Epoch 8/15
18/18 [==

In [14]:
gru_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, None, 80, 8  0           []                               
                                0, 3)]                                                            
                                                                                                  
 model_2 (Functional)           (None, None, 10, 10  170416      ['input_3[0][0]']                
                                , 64)                                                             
                                                                                                  
 time_distributed_43 (TimeDistr  (None, None, 6400)  0           ['model_2[0][0]']                
 ibuted)                                                                                    

In [15]:
gru_model.save_weights('/content/drive/MyDrive/video_classification_rnn/weights/gru_model.h5')

#### 03- LSTM

In [16]:
models = Models((frame_height, frame_width), max_seq_len)
lstm_model = models.LSTM_model()

lstm_model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

lstm_model.fit([X_train, mask_train], Y_train, validation_data=[[X_val, mask_val], Y_val], batch_size=3, epochs=epochs)


Epoch 1/15
18/18 [==============================] - 40s 2s/step - loss: 1.1698 - accuracy: 0.4630 - val_loss: 0.8004 - val_accuracy: 0.5000
Epoch 2/15
18/18 [==============================] - 33s 2s/step - loss: 0.7429 - accuracy: 0.5556 - val_loss: 0.7682 - val_accuracy: 0.5000
Epoch 3/15
18/18 [==============================] - 33s 2s/step - loss: 0.7575 - accuracy: 0.5370 - val_loss: 0.7892 - val_accuracy: 0.4286
Epoch 4/15
18/18 [==============================] - 33s 2s/step - loss: 0.7417 - accuracy: 0.5185 - val_loss: 0.7658 - val_accuracy: 0.6429
Epoch 5/15
18/18 [==============================] - 33s 2s/step - loss: 0.7479 - accuracy: 0.5741 - val_loss: 0.7765 - val_accuracy: 0.4286
Epoch 6/15
18/18 [==============================] - 33s 2s/step - loss: 0.6989 - accuracy: 0.6296 - val_loss: 0.7941 - val_accuracy: 0.5000
Epoch 7/15
18/18 [==============================] - 33s 2s/step - loss: 0.6594 - accuracy: 0.6667 - val_loss: 0.7802 - val_accuracy: 0.3571
Epoch 8/15
18/18 [==

In [17]:
lstm_model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, None, 80, 8  0           []                               
                                0, 3)]                                                            
                                                                                                  
 model_4 (Functional)           (None, None, 10, 10  170416      ['input_5[0][0]']                
                                , 64)                                                             
                                                                                                  
 time_distributed_65 (TimeDistr  (None, None, 6400)  0           ['model_4[0][0]']                
 ibuted)                                                                                    

In [18]:
lstm_model.save_weights('/content/drive/MyDrive/video_classification_rnn/weights/lstm_model.h5')